In [3]:
# If needed:
# !pip -q install yfinance pandas numpy

from __future__ import annotations

from dataclasses import dataclass
from typing import Iterable
import pandas as pd
import numpy as np

@dataclass(frozen=True)
class Tickers:
    # Indices
    DJI: str = "^DJI"     # DJIA (USD)
    FTSE: str = "^FTSE"   # FTSE 100 (GBP)
    FCHI: str = "^FCHI"   # CAC 40 (EUR)
    N225: str = "^N225"   # Nikkei 225 (JPY)

    # FX building blocks (Yahoo Finance)
    GBPUSD: str = "GBPUSD=X"  # USD per 1 GBP
    EURUSD: str = "EURUSD=X"  # USD per 1 EUR
    USDMXN: str = "MXN=X"     # MXN per 1 USD (USD/MXN)
    USDJPY: str = "JPY=X"     # JPY per 1 USD (USD/JPY)

def _download_adj_close(tickers: Iterable[str], start: str, end: str) -> pd.DataFrame:
    import yfinance as yf

    tickers = list(tickers)
    data = yf.download(
        tickers=tickers,
        start=start,
        end=end,          # end is typically exclusive in yfinance
        interval="1d",
        auto_adjust=False,
        progress=False,
        group_by="column",
        actions=False,
    )

    if data.empty:
        raise ValueError("Empty download. Check tickers/dates/connectivity.")

    if isinstance(data.columns, pd.MultiIndex):
        if "Adj Close" in data.columns.get_level_values(0):
            out = data["Adj Close"].copy()
        elif "Close" in data.columns.get_level_values(0):
            out = data["Close"].copy()
        else:
            raise ValueError("Neither 'Adj Close' nor 'Close' found.")
    else:
        col = "Adj Close" if "Adj Close" in data.columns else "Close"
        out = data[[col]].copy()
        out.columns = tickers

    out.index = pd.to_datetime(out.index)
    return out.sort_index()

def build_dataset(start: str, end: str) -> pd.DataFrame:
    tk = Tickers()

    # 1) Index levels (Adj Close)
    idx_raw = _download_adj_close([tk.DJI, tk.FTSE, tk.FCHI, tk.N225], start, end)
    idx = idx_raw.rename(
        columns={
            tk.DJI: "DJIA_USD",
            tk.FTSE: "FTSE100_GBP",
            tk.FCHI: "CAC40_EUR",
            tk.N225: "NIKKEI225_JPY",
        }
    )

    # 2) FX legs
    fx_raw = _download_adj_close([tk.GBPUSD, tk.EURUSD, tk.USDMXN, tk.USDJPY], start, end)
    fx_raw = fx_raw.rename(
        columns={
            tk.GBPUSD: "GBPUSD",   # USD per GBP
            tk.EURUSD: "EURUSD",   # USD per EUR
            tk.USDMXN: "USDMXN",   # MXN per USD
            tk.USDJPY: "USDJPY",   # JPY per USD
        }
    )

    # 3) Build desired FX cross-rates
    fx = pd.DataFrame(index=fx_raw.index)
    fx["USD_per_USD"] = 1.0
    fx["USD_per_GBP"] = fx_raw["GBPUSD"]
    fx["USD_per_EUR"] = fx_raw["EURUSD"]
    fx["USD_per_JPY"] = 1.0 / fx_raw["USDJPY"]                # USD/JPY inverted -> USD per JPY

    fx["MXN_per_USD"] = fx_raw["USDMXN"]
    fx["MXN_per_GBP"] = fx_raw["GBPUSD"] * fx_raw["USDMXN"]    # (USD/GBP)*(MXN/USD)=MXN/GBP
    fx["MXN_per_EUR"] = fx_raw["EURUSD"] * fx_raw["USDMXN"]    # (USD/EUR)*(MXN/USD)=MXN/EUR
    fx["MXN_per_JPY"] = fx_raw["USDMXN"] / fx_raw["USDJPY"]    # (MXN/USD)/(JPY/USD)=MXN/JPY

    # 4) Keep only dates where ALL indices traded, then require ALL FX too
    idx_common = idx.dropna(how="any")
    out = idx_common.join(fx, how="inner").dropna(how="any")

    out = out[~out.index.duplicated(keep="first")].sort_index()
    return out

# --- Run (2005-01-01 through "today") ---
start = "2005-01-01"
# yfinance end is usually exclusive; use tomorrow to include today if available
end = (pd.Timestamp.today().normalize() + pd.Timedelta(days=1)).strftime("%Y-%m-%d")

df = build_dataset(start, end)

csv_path = "indices_fx_aligned_2005_to_today.csv"
df.to_csv(csv_path, index_label="Date")

df.head(), df.tail(), df.shape, csv_path


(                DJIA_USD    CAC40_EUR  FTSE100_GBP  NIKKEI225_JPY  \
 Date                                                                
 2005-01-04  10630.780273  3863.300049  4847.000000   11517.750000   
 2005-01-05  10597.830078  3829.360107  4806.000000   11437.519531   
 2005-01-06  10622.879883  3856.479980  4824.299805   11492.259766   
 2005-01-07  10603.959961  3877.959961  4854.100098   11433.240234   
 2005-01-11  10556.219727  3848.989990  4818.700195   11539.990234   
 
             USD_per_USD  USD_per_GBP  USD_per_EUR  USD_per_JPY  MXN_per_USD  \
 Date                                                                          
 2005-01-04          1.0     1.883594     1.328198     0.009584       11.347   
 2005-01-05          1.0     1.885512     1.328004     0.009622       11.330   
 2005-01-06          1.0     1.876490     1.318305     0.009534       11.358   
 2005-01-07          1.0     1.871293     1.306097     0.009534       11.246   
 2005-01-11          1.0    

In [4]:
df

,DJIA_USD,CAC40_EUR,FTSE100_GBP,NIKKEI225_JPY,USD_per_USD,USD_per_GBP,USD_per_EUR,USD_per_JPY,MXN_per_USD,MXN_per_GBP,MXN_per_EUR,MXN_per_JPY
Date,,,,,,,,,,,,
2005-01-04,10630.780273,3863.300049,4847.000000,11517.750000,1.0,1.883594,1.328198,0.009584,11.347000,21.373140,15.071058,0.108750
2005-01-05,10597.830078,3829.360107,4806.000000,11437.519531,1.0,1.885512,1.328004,0.009622,11.330000,21.362848,15.046281,0.109016
2005-01-06,10622.879883,3856.479980,4824.299805,11492.259766,1.0,1.876490,1.318305,0.009534,11.358000,21.313168,14.973304,0.108285
2005-01-07,10603.959961,3877.959961,4854.100098,11433.240234,1.0,1.871293,1.306097,0.009534,11.246000,21.044557,14.688365,0.107217
2005-01-11,10556.219727,3848.989990,4818.700195,11539.990234,1.0,1.878605,1.311699,0.009669,11.177000,20.997163,14.660860,0.108074
...,...,...,...,...,...,...,...,...,...,...,...,...
2026-01-09,49504.070312,8362.089844,10124.599609,51939.890625,1.0,1.343801,1.165787,0.006374,17.962299,24.137748,20.940206,0.114497
2026-01-13,49191.988281,8347.200195,10137.400391,53549.160156,1.0,1.346566,1.166698,0.006330,17.909100,24.115777,20.894508,0.113359
2026-01-14,49149.628906,8330.969727,10184.400391,54341.230469,1.0,1.342678,1.164253,0.006282,17.823601,23.931362,20.751176,0.111972
